## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [182]:
path = '../04-filtrage/output/'
acteur = 'ciusss_centresud'
tag = ''

if tag:
    csv_file = acteur + '_' + tag + '_significant-collocations.csv'

else:
    csv_file = acteur + '_significant-collocations.csv'

### **Lire le vocabulaire** (termes retenus au prétraitement)

In [183]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f).drop(columns = ['Unnamed: 0'])
    # On va traiter les apostrophes et les parenthèses avec le underscore ("_") ; le tokenizer de sklearn ne les aime pas 
    csv['Terme formatté'] = csv["Terme"].apply(lambda x: str(x).replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_"))

csv

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté
0,ciusss_centresud,services,NOM,3252,622,-,False,False,services
1,ciusss_centresud,hôpital,NOM,1724,262,-,False,True,hôpital
2,ciusss_centresud,médecin,NOM,1590,364,-,False,True,médecin
3,ciusss_centresud,centre,NOM,1348,448,-,False,False,centre
4,ciusss_centresud,carte,NOM,902,212,-,False,False,carte
...,...,...,...,...,...,...,...,...,...
1615,ciusss_centresud,itss et contraception,NOM KON NOM,16,16,286.332206151949,False,False,itss et contraception
1616,ciusss_centresud,équité salariale,NOM ADJ,16,4,288.02239872416635,True,False,équité salariale
1617,ciusss_centresud,consommation d'alcool soutien,NOM PRP NOM NOM,16,16,197.07839676405885,False,False,consommation d_alcool soutien
1618,ciusss_centresud,crlb et ird,NOM KON ADJ,16,16,294.37503282131814,False,False,crlb et ird


In [184]:
vocabulaire = [t.lower() for t in list(csv['Terme formatté'])]

In [185]:
print('On a un vocabulaire de {} formes.'.format(len(vocabulaire)))

On a un vocabulaire de 1620 formes.


In [186]:
vocabulaire

['services',
 'hôpital',
 'médecin',
 'centre',
 'carte',
 'accès',
 'évaluation',
 'ciusss',
 'santé',
 'clsc',
 'demande',
 'clinique',
 'soins',
 'verdun',
 'heures',
 'carte d_hôpital',
 'hôpital du ciusss',
 'carte d_hôpital du ciusss',
 'vie',
 'réadaptation',
 'proches',
 'enfant',
 'requête',
 'hébergement',
 'lieu',
 'notre_dame',
 'usagers',
 'québec',
 'poste',
 'hôpital notre_dame',
 'programme',
 'consultation',
 'infirmière',
 'accès aux services',
 'ouverture',
 'déficience',
 'heures d_ouverture',
 'hôpital de verdun',
 'ans',
 'maladie',
 'télécopieur',
 'visiteurs',
 'problèmes',
 'aide',
 'aidants',
 'service',
 'assurance',
 'requête du médecin',
 'services offerts',
 'cliniques',
 'services sont offerts',
 'soutien',
 'déficience physique',
 'main',
 'travailleur social',
 'externes',
 'travailleur',
 'assurance maladie',
 'plaintes',
 'travail',
 'cliniques externes',
 'besoins',
 'famille',
 'faubourgs',
 'enfants',
 'aînés',
 'suivi',
 'milieu',
 'chsld',
 'cart

### **Lire le corpus**

In [187]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

In [188]:
text = text[:round(len(text))]

nb_docs = len(text)

print("On a donc un corpus de {} documents.".format(nb_docs))

On a donc un corpus de 854 documents.


### **Nettoyage**

In [189]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&•►*+,;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t).replace("' ", "'" ) for t in corpus]
corpus = [re.sub(phones, ' STOP ', t) for t in corpus]
corpus = [re.sub(postals, ' STOP ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]
corpus = [t.replace("'", "_").replace("(", "_").replace(")", "_").replace("-", "_") for t in corpus]


On va commencer par utiliser le CountVectorizer pour valider que l'implémentation de sklearn arrive bien au même compte que nous 

In [190]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=vocabulaire, ngram_range=(1,10), token_pattern='\w+')
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()

features_names = vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(X.T.todense(), index=features_names, columns=corpus_index).transpose()

In [191]:
csv['TF (sklearn)'] = 0
for t in vocabulaire:
    csv.loc[csv['Terme formatté'] == t, 'TF (sklearn)'] = df[t].sum()


csv['DF (sklearn)'] = 0
for t in vocabulaire:
    freqdoc = len(df[df[t] != 0])
    csv.loc[csv['Terme formatté'] == t, 'DF (sklearn)'] = freqdoc
csv

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn)
0,ciusss_centresud,services,NOM,3252,622,-,False,False,services,3398,622
1,ciusss_centresud,hôpital,NOM,1724,262,-,False,True,hôpital,442,232
2,ciusss_centresud,médecin,NOM,1590,364,-,False,True,médecin,1578,326
3,ciusss_centresud,centre,NOM,1348,448,-,False,False,centre,1344,380
4,ciusss_centresud,carte,NOM,902,212,-,False,False,carte,902,212
...,...,...,...,...,...,...,...,...,...,...,...
1615,ciusss_centresud,itss et contraception,NOM KON NOM,16,16,286.332206151949,False,False,itss et contraception,16,16
1616,ciusss_centresud,équité salariale,NOM ADJ,16,4,288.02239872416635,True,False,équité salariale,8,4
1617,ciusss_centresud,consommation d'alcool soutien,NOM PRP NOM NOM,16,16,197.07839676405885,False,False,consommation d_alcool soutien,16,16
1618,ciusss_centresud,crlb et ird,NOM KON ADJ,16,16,294.37503282131814,False,False,crlb et ird,16,16


In [192]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import RegexpTokenizer

#toknizer = RegexpTokenizer()

# max_df : ignore words that appear in 85% of documents, 
# min df:  ignore words that appear in less than 1% of documents 
# vocabulary = vocabulaire

# Sans utiliser le vocabulaire
# tfidf = TfidfVectorizer(min_df=0.1, stop_words=None, ngram_range=(2,4), max_df=0.85, use_idf=True)

# vocabulary = vocabulaire
tfidf_vectorizer = TfidfVectorizer(vocabulary=vocabulaire, ngram_range=(1,10), lowercase=False, token_pattern='\w+')
tfidf = tfidf_vectorizer.fit_transform(corpus)

In [193]:
features_names = tfidf_vectorizer.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()

In [194]:
terms_tfidf = {term: df[term].max() for term in df}
zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}

In [195]:
zeros

{'hôpital du ciusss': 0.0,
 'assurance maladie': 0.0,
 'ouverture et lieu heures': 0.0,
 'ouverture et lieu': 0.0,
 'hôpital du ciusss du centre_sud_de_l_île_de_montréal carte': 0.0,
 'centre de rendez': 0.0,
 'secteur jaune': 0.0,
 'secteur mauve': 0.0,
 'fax': 0.0,
 'accès au dossier de l_usager': 0.0,
 'ostéointégration': 0.0,
 'assistance à la motricité': 0.0,
 'octroi': 0.0,
 'octroi de la tarification': 0.0,
 'accompagnement aux plaintes _caap_': 0.0,
 'accompagnement aux plaintes': 0.0,
 'caap': 0.0,
 'accès aux proches aidants': 0.0,
 'accès aux proches': 0.0,
 'avancement': 0.0,
 'octroi de la tarification wifi public': 0.0,
 'octroi de la tarification wifi': 0.0,
 'achat': 0.0,
 'usager sans rendez': 0.0,
 'résident sans rendez': 0.0,
 'halte': 0.0,
 'horaire des rendez': 0.0,
 'assurance automobile': 0.0,
 'assurance automobile du québec': 0.0,
 'orchestre qui dirige': 0.0,
 'orchestre': 0.0,
 'médecin sans rendez': 0.0,
 'unité épaulent': 0.0,
 'approche milieu': 0.0,
 'app

In [196]:
len(zeros) / len(vocabulaire) *100

6.049382716049383

In [197]:
terms_tfidf

{'services': 0.3472631041476018,
 'hôpital': 0.22588022396548252,
 'médecin': 0.34836438580095597,
 'centre': 0.3442539168714698,
 'carte': 0.31292514279843114,
 'accès': 1.0,
 'évaluation': 0.15216920144575655,
 'ciusss': 0.2096042669513844,
 'santé': 0.5010197359127341,
 'clsc': 0.7116120255755354,
 'demande': 0.2928438575286933,
 'clinique': 0.33576454432525266,
 'soins': 0.42354307226596516,
 'verdun': 0.17661102712431329,
 'heures': 0.221232761482838,
 'carte d_hôpital': 0.2539919642874409,
 'hôpital du ciusss': 0.0,
 'carte d_hôpital du ciusss': 0.2551207824901067,
 'vie': 0.30161237400338103,
 'réadaptation': 0.47368176296550646,
 'proches': 0.5954960268847945,
 'enfant': 0.4945037043029923,
 'requête': 0.3623942450647831,
 'hébergement': 0.5897353043078808,
 'lieu': 0.15750998415211556,
 'notre_dame': 0.2438702902198315,
 'usagers': 0.33648770058330685,
 'québec': 0.32019435180014844,
 'poste': 0.1827937968007939,
 'hôpital notre_dame': 0.15180079876606226,
 'programme': 0.3878

In [198]:
terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme formatté', 'TF-IDF'])

In [199]:
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

terms_weighted = terms_weighted.drop_duplicates(keep='first')
terms_weighted

,Terme formatté,TF-IDF
5,accès,1.000000
153,vaccination,0.907513
447,spectre,0.903241
907,vaccin,0.899813
386,adoption,0.875150
...,...,...
573,accès aux proches aidants,0.000000
1528,accès clinique,0.000000
726,résident sans rendez,0.000000
111,ouverture et lieu,0.000000


In [200]:
terms_weighted = pd.merge(csv, terms_weighted, on='Terme formatté').drop_duplicates(
  subset = ['Terme', 'Fréquence (TF)'],
  keep = 'first').reset_index(drop = True)

terms_weighted

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,ciusss_centresud,services,NOM,3252,622,-,False,False,services,3398,622,0.347263
1,ciusss_centresud,hôpital,NOM,1724,262,-,False,True,hôpital,442,232,0.225880
2,ciusss_centresud,médecin,NOM,1590,364,-,False,True,médecin,1578,326,0.348364
3,ciusss_centresud,centre,NOM,1348,448,-,False,False,centre,1344,380,0.344254
4,ciusss_centresud,carte,NOM,902,212,-,False,False,carte,902,212,0.312925
...,...,...,...,...,...,...,...,...,...,...,...,...
1615,ciusss_centresud,itss et contraception,NOM KON NOM,16,16,286.332206151949,False,False,itss et contraception,16,16,0.348877
1616,ciusss_centresud,équité salariale,NOM ADJ,16,4,288.02239872416635,True,False,équité salariale,8,4,0.514967
1617,ciusss_centresud,consommation d'alcool soutien,NOM PRP NOM NOM,16,16,197.07839676405885,False,False,consommation d_alcool soutien,16,16,0.116204
1618,ciusss_centresud,crlb et ird,NOM KON ADJ,16,16,294.37503282131814,False,False,crlb et ird,16,16,0.139772


## **OKapi BM25**
https://hal.archives-ouvertes.fr/hal-00760158 

In [201]:
from rank_bm25 import BM25Okapi

In [202]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")

bm25 = BM25Okapi([tokenizer.tokenize(t) for t in corpus])

In [203]:
vocabulaire = csv["Terme"].dropna().tolist()

vocabulaire

['services',
 'hôpital',
 'médecin',
 'centre',
 'carte',
 'accès',
 'évaluation',
 'ciusss',
 'santé',
 'clsc',
 'demande',
 'clinique',
 'soins',
 'verdun',
 'heures',
 "carte d'hôpital",
 'hôpital du ciusss',
 "carte d'hôpital du ciusss",
 'vie',
 'réadaptation',
 'proches',
 'enfant',
 'requête',
 'hébergement',
 'lieu',
 'notre-dame',
 'usagers',
 'québec',
 'poste',
 'hôpital notre-dame',
 'programme',
 'consultation',
 'infirmière',
 'accès aux services',
 'ouverture',
 'déficience',
 "heures d'ouverture",
 'hôpital de verdun',
 'ans',
 'maladie',
 'télécopieur',
 'visiteurs',
 'problèmes',
 'aide',
 'aidants',
 'service',
 'assurance',
 'requête du médecin',
 'services offerts',
 'cliniques',
 'services sont offerts',
 'soutien',
 'déficience physique',
 'main',
 'travailleur social',
 'externes',
 'travailleur',
 'assurance maladie',
 'plaintes',
 'travail',
 'cliniques externes',
 'besoins',
 'famille',
 'faubourgs',
 'enfants',
 'aînés',
 'suivi',
 'milieu',
 'chsld',
 "cart

In [204]:
#tokenizer = RegexpTokenizer(r"\w\'|\w+")
## Revoir ça
tokenized_queries = [tokenizer.tokenize(str(t)) for t in vocabulaire]

features_names = [t for t in set(vocabulaire)]
corpus_index = [corpus.index(n) for n in corpus]

tab = [bm25.get_scores(query) for query in tokenized_queries]
df = pd.DataFrame(tab, index=features_names, columns=corpus_index).transpose()

In [205]:
print(len(tokenized_queries))

tokenized_queries

1620


[['services'],
 ['hôpital'],
 ['médecin'],
 ['centre'],
 ['carte'],
 ['accès'],
 ['évaluation'],
 ['ciusss'],
 ['santé'],
 ['clsc'],
 ['demande'],
 ['clinique'],
 ['soins'],
 ['verdun'],
 ['heures'],
 ['carte', "d'", 'hôpital'],
 ['hôpital', 'du', 'ciusss'],
 ['carte', "d'", 'hôpital', 'du', 'ciusss'],
 ['vie'],
 ['réadaptation'],
 ['proches'],
 ['enfant'],
 ['requête'],
 ['hébergement'],
 ['lieu'],
 ['notre-dame'],
 ['usagers'],
 ['québec'],
 ['poste'],
 ['hôpital', 'notre-dame'],
 ['programme'],
 ['consultation'],
 ['infirmière'],
 ['accès', 'aux', 'services'],
 ['ouverture'],
 ['déficience'],
 ['heures', "d'", 'ouverture'],
 ['hôpital', 'de', 'verdun'],
 ['ans'],
 ['maladie'],
 ['télécopieur'],
 ['visiteurs'],
 ['problèmes'],
 ['aide'],
 ['aidants'],
 ['service'],
 ['assurance'],
 ['requête', 'du', 'médecin'],
 ['services', 'offerts'],
 ['cliniques'],
 ['services', 'sont', 'offerts'],
 ['soutien'],
 ['déficience', 'physique'],
 ['main'],
 ['travailleur', 'social'],
 ['externes'],
 [

In [206]:
df

,distance d'un mètre lors de la prise,soirée,lucie-bruneau (crlb),douleur,abus,physiatre orthopédiste neurologue rhumatologue,québec (saaq),accompagnement des femmes enceintes en difficultés,environnement,madame,...,île,note,communauté,unité de courte,lsq,cours prénataux,chsld demande d'accès,amplificateur,soins,lucie-bruneau
0,2.253282,0.000000,0.495294,0.321742,0.0,0.000000,1.783394,0.000000,0.000000,0.000000,...,2.639671,2.639671,0.0,0.748225,0.0,2.378626,0.0,0.000000,6.092119,0.0
1,2.323144,0.000000,0.766689,0.000000,0.0,0.180286,0.794196,0.486052,0.000000,0.000000,...,4.751133,3.820156,0.0,0.000000,0.0,2.596510,0.0,0.930977,2.596510,0.0
2,2.329314,0.000000,0.469026,0.000000,0.0,0.000000,1.191764,0.000000,0.578596,1.311373,...,2.878903,2.878903,0.0,0.000000,0.0,2.751386,0.0,0.000000,2.751386,0.0
3,2.250020,0.581056,0.977903,0.130256,0.0,0.000000,1.857009,0.000000,0.243537,0.794463,...,4.347556,4.347556,0.0,0.429254,0.0,2.666920,0.0,0.000000,2.666920,0.0
4,1.708928,0.000000,0.000000,0.000000,0.0,0.000000,0.907643,0.000000,0.306156,0.000000,...,5.106048,2.854313,0.0,0.000000,0.0,2.516926,0.0,2.251735,2.516926,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.627613,2.627613,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
149,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.627613,2.627613,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
149,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.627613,2.627613,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0
149,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.627613,2.627613,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [207]:
#df.to_csv(base_path + titre + '_matrice-OkapiBM25.csv') # Si on veut avoir la matrice (mais le fichier peut être très volumineux)

In [208]:
terms_okapi = {term: df[term].max() for term in df}

In [209]:
terms_okapi

{"distance d'un mètre lors de la prise": 2.6681103014651213,
 'soirée': 1.8892373317454885,
 'lucie-bruneau (crlb)': 1.096648817765348,
 'douleur': 0.5053542135190763,
 'abus': 2.383793483656998,
 'physiatre orthopédiste neurologue rhumatologue': 0.5416427375716485,
 'québec (saaq)': 2.6988096357186095,
 'accompagnement des femmes enceintes en difficultés': 1.4961086304589226,
 'environnement': 0.9146084691017217,
 'madame': 3.0952784797277033,
 "chaussures orthétiques et d'appareillages": 0.5283831999928823,
 'griffintown': 0.5459789401766725,
 'durée gériatrique': 2.6824840137631725,
 'vie respectueux': 1.2114822850797675,
 'gériatre': 0.9362171444646419,
 'visitation': 4.1427740942976214,
 'infirmières': 6.141352594543764,
 'maison de naissance jeanne-mance': 8.394889357095897,
 'soutien urgent en cas de crise': 2.978589395622237,
 'octroi de la tarification': 2.956678872365637,
 'laurier': 3.3969166149090526,
 'notre-dame déroulement': 3.2876584511758358,
 'visites des familles': 2

In [210]:
tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
tab

,Terme,OkapiBM25
0,distance d'un mètre lors de la prise,2.668110
1,soirée,1.889237
2,lucie-bruneau (crlb),1.096649
3,douleur,0.505354
4,abus,2.383793
...,...,...
1615,cours prénataux,14.504706
1616,chsld demande d'accès,21.933486
1617,amplificateur,8.125364
1618,soins,14.708467


In [211]:
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

tab

,Terme,OkapiBM25
176,clinique de cognition,39.602486
1537,problèmes liés,35.998919
1002,usagers du sud-ouest-verdun,28.025181
1037,accès aux services secteurs des faubourgs,28.008228
1412,gynécologie,26.848322
...,...,...
754,fin de semaine,0.000000
1208,ciusss du centre-sud-de-l'île-de-montréal carte,0.000000
789,parent,0.000000
1205,centre d'assistance et d'accompagnement aux pl...,0.000000


In [212]:
terms_weighted

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF
0,ciusss_centresud,services,NOM,3252,622,-,False,False,services,3398,622,0.347263
1,ciusss_centresud,hôpital,NOM,1724,262,-,False,True,hôpital,442,232,0.225880
2,ciusss_centresud,médecin,NOM,1590,364,-,False,True,médecin,1578,326,0.348364
3,ciusss_centresud,centre,NOM,1348,448,-,False,False,centre,1344,380,0.344254
4,ciusss_centresud,carte,NOM,902,212,-,False,False,carte,902,212,0.312925
...,...,...,...,...,...,...,...,...,...,...,...,...
1615,ciusss_centresud,itss et contraception,NOM KON NOM,16,16,286.332206151949,False,False,itss et contraception,16,16,0.348877
1616,ciusss_centresud,équité salariale,NOM ADJ,16,4,288.02239872416635,True,False,équité salariale,8,4,0.514967
1617,ciusss_centresud,consommation d'alcool soutien,NOM PRP NOM NOM,16,16,197.07839676405885,False,False,consommation d_alcool soutien,16,16,0.116204
1618,ciusss_centresud,crlb et ird,NOM KON ADJ,16,16,294.37503282131814,False,False,crlb et ird,16,16,0.139772


In [213]:
tab = pd.merge(terms_weighted, tab, on="Terme")
tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

In [214]:
base_path = '../05-transformation/'

In [215]:
if tag:
    file_path = base_path + acteur + '_' + tag + '_weighting_OKapiBM25.csv'

else: 
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
tab.to_csv(file_path)

In [216]:
tab

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,Terme formatté,TF (sklearn),DF (sklearn),TF-IDF,OkapiBM25
1152,ciusss_centresud,clinique de cognition,NOM PRP NOM,24,6,268.52719786845165,False,False,clinique de cognition,24,6,0.382398,39.602486
627,ciusss_centresud,problèmes liés,NOM VER:pper,42,30,411.76460754252577,False,False,problèmes liés,42,30,0.161476,35.998919
1122,ciusss_centresud,usagers du sud-ouest-verdun,NOM PRP:det ADJ,24,24,380.6061081195228,False,False,usagers du sud_ouest_verdun,24,24,0.128925,28.025181
481,ciusss_centresud,accès aux services secteurs des faubourgs,NOM PRP:det NOM NOM PRP:det NOM,58,58,831.6896736700168,False,False,accès aux services secteurs des faubourgs,58,58,0.133540,28.008228
1280,ciusss_centresud,gynécologie,NOM,20,14,-,False,True,gynécologie,20,14,0.264308,26.848322
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,ciusss_centresud,moments,NOM,20,16,-,False,False,moments,20,16,0.139046,0.000000
928,ciusss_centresud,langue,NOM,30,26,-,False,True,langue,30,26,0.183073,0.000000
1300,ciusss_centresud,conditions de vie,NOM PRP NOM,20,14,198.9754976932994,True,True,conditions de vie,20,14,0.266079,0.000000
954,ciusss_centresud,programme de réadaptation au travail,NOM PRP NOM PRP:det NOM,30,10,422.741456750425,False,False,programme de réadaptation au travail,30,10,0.421468,0.000000
